In [20]:
using Catan
using CatanLearning
using Random
using CSV
using MLJ
using DataFrames
import DataFramesMeta as DFM
using DelimitedFiles
import MLJModelInterface
const MMI = MLJModelInterface

MLJModelInterface

In [21]:
function naive_model_proba(X_in)
    X = X_in[1]
    #println("analyzing $X")
    unscaled_feature = X[!,:CountSettlement] + X[!,:CountCity] + 2*(X[!,:HasLargestArmy] + X[!,:HasLongestRoad])
    return min.(unscaled_feature ./ 10, 1.0)
end

MMI.@mlj_model mutable struct NaivePublicVictoryPoints <: MLJModelInterface.Deterministic
end

MMI.fit(::NaivePublicVictoryPoints, verbosity, data...) = (nothing, nothing, nothing)
MMI.predict(model::NaivePublicVictoryPoints, fitresult, new_data...) = naive_model_proba(new_data)

model = NaivePublicVictoryPoints()
naive_machine = machine(model, nothing, nothing) |> MLJ.fit!

trained Machine; caches model-specific representations of data
  model: NaivePublicVictoryPoints()
  args: 
    1:	Source @616 ⏎ Nothing
    2:	Source @900 ⏎ Nothing


In [30]:
config_file = "feature_generation.toml"
(configs, player_configs, logger) = Catan.parse_configs(config_file)
rf_machine = CatanLearning.try_load_public_model_from_csv(player_configs)
#rf_machine = CatanLearning.train_model_from_csv(CatanLearning.load_tree_model(), player_configs["PUBLIC_FEATURES"], num_tuning_iterations = 1)

Savefile set to savefile.txt
Configs loaded from feature_generation.toml


trained Machine; caches model-specific representations of data
  model: RandomForestClassifier(max_depth = -1, …)
  args: 


In [23]:
test_data_csv = "./data/test_public_features.csv"
df = CatanLearning.load_typed_features_from_csv(test_data_csv)
y, X = unpack(df, ==(:WonGame));
typeof(X)
for col in names(X)
    println(col)
end

CountSettlement
CountTotalSettlement
CountCity
CountRoad
SumWoodDiceWeight
SumBrickDiceWeight
SumPastureDiceWeight
SumStoneDiceWeight
SumGrainDiceWeight
CountPortWood
CountPortBrick
CountPortPasture
CountPortStone
CountPortGrain
CountTotalWood
CountTotalBrick
CountTotalPasture
CountTotalStone
CountTotalGrain
CountDevCardsKnight
CountDevCardsMonopoly
CountDevCardsYearOfPlenty
CountDevCardsRoadBuilding
CountDevCardsVictoryPoint
HasLargestArmy
HasLongestRoad


In [26]:
rf_res = mode.(MLJ.predict(rf_machine, X))


4120-element CategoricalArrays.CategoricalArray{Bool,1,UInt32}:
 false
 true
 false
 false
 false
 false
 true
 false
 false
 false
 ⋮
 false
 false
 false
 true
 false
 false
 true
 false
 false

In [27]:
naive_res = mode.(MLJ.predict(naive_machine, X))

4120-element Vector{Float64}:
 0.3
 0.4
 0.2
 0.2
 0.2
 0.5
 0.4
 0.2
 0.2
 0.5
 ⋮
 0.4
 0.5
 0.2
 0.7
 0.5
 0.2
 0.6
 0.2
 0.2

In [ ]:
#MLJ.predict(naive_machine, X)
#MLJ.save(naive_machine, "../CatanLearning/data/naive_public_model.jls")

"../CatanLearning/data/naive_public_model.jls"

In [45]:
mach = rf_machine

mcc(mach, X, y)

MethodError: MethodError: no method matching length(::Machine{MLJDecisionTreeInterface.RandomForestClassifier, MLJDecisionTreeInterface.RandomForestClassifier, true})

Closest candidates are:
  length(!Matched::Distributions.DirichletStats)
   @ Distributions ~/.julia/packages/Distributions/nmUhn/src/multivariate/dirichlet.jl:186
  length(!Matched::CSV.Chunks)
   @ CSV ~/.julia/packages/CSV/XLcqT/src/chunks.jl:88
  length(!Matched::Combinatorics.IntegerPartitions)
   @ Combinatorics ~/.julia/packages/Combinatorics/Udg6X/src/partitions.jl:17
  ...


In [32]:
report(rf_machine)

(features = [:CountSettlement, :CountTotalSettlement, :CountCity, :CountRoad, :SumWoodDiceWeight, :SumBrickDiceWeight, :SumPastureDiceWeight, :SumStoneDiceWeight, :SumGrainDiceWeight, :CountPortWood  …  :CountTotalPasture, :CountTotalStone, :CountTotalGrain, :CountDevCardsKnight, :CountDevCardsMonopoly, :CountDevCardsYearOfPlenty, :CountDevCardsRoadBuilding, :CountDevCardsVictoryPoint, :HasLargestArmy, :HasLongestRoad],)